<a href="https://colab.research.google.com/github/moodlep/basic_dqn/blob/master/mlc_dqn_atari.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Do we need this?? 

In [ ]:
!pip install ale-py

     |████████████████████████████████| 1.6 MB 4.2 MB/s 


In [ ]:
from ale_py import ALEInterface

ale = ALEInterface()

In [ ]:
from ale_py.roms import Breakout

ale.loadROM(Breakout)

Below taken from [link](https://colab.research.google.com/github/GiannisMitr/DQN-Atari-Breakout/blob/master/dqn_atari_breakout.ipynb)


# Install Atari ROMs 

In [ ]:
#Make necessary imports

import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import urllib.request
urllib.request.urlretrieve('http://www.atarimania.com/roms/Roms.rar','Roms.rar')
!pip install unrar
!unrar x Roms.rar
!mkdir rars
!mv HC\ ROMS.zip   rars
!mv ROMS.zip  rars
!python -m atari_py.import_roms rars

In [ ]:
# Test a basic env: 
import gym
env = gym.make('Breakout-v4')
env.observation_space.shape, env.action_space.n

((210, 160, 3), 18)

## SB3 has some nice wrappers we can use, taken from cleanrl

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
# taken from https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/atari/dqn_atari.py
import gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from gym.spaces import Discrete
from gym.wrappers import Monitor
from stable_baselines3.common.atari_wrappers import (
    ClipRewardEnv,
    EpisodicLifeEnv,
    FireResetEnv,
    MaxAndSkipEnv,
    NoopResetEnv,
    WarpFrame,
)
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack


In [ ]:
# taken from https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/atari/dqn_atari.py
def make_env(gym_id, idx):
    def thunk():
        env = gym.make(gym_id)
        env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        # if args.capture_video:
        #     if idx == 0:
        #         env = Monitor(env, f"videos/{experiment_name}")
        env = EpisodicLifeEnv(env)
        if "FIRE" in env.unwrapped.get_action_meanings():
            env = FireResetEnv(env)
        env = WarpFrame(env, width=84, height=84)
        env = ClipRewardEnv(env)
        # env.seed(seed)
        # env.action_space.seed(seed)
        # env.observation_space.seed(seed)
        return env

    return thunk

In [ ]:
envs = VecFrameStack(
        DummyVecEnv([make_env('Breakout-v4', 0)]),
        4,
    )

In [ ]:
envs.observation_space.shape, envs.action_space.n

((84, 84, 4), 4)

In [ ]:
envs.observation_space.shape

# Coding DQN with CNNs

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class DQN_Network(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DQN_Network, self).__init__()

        self.state_dim = state_dim
        self.action_dim = action_dim

        # create the dqn and target network architecture
        self.network = nn.Sequential(
            nn.Conv2d(4, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2), 
            nn.ReLU(), 
            nn.Conv2d(64, 64, 3, stride=1), 
            nn.ReLU(), 
            nn.Flatten(),
            nn.Linear(64*7*7, 256),
            nn.ReLU(),
            nn.Linear(256, self.action_dim)

        )

    def forward(self, input):
        
        state_t = torch.FloatTensor(input).permute(2, 0, 1).unsqueeze(0)
        print(state_t.shape, type(state_t))

        q_values = self.network(state_t)
        return q_values


In [ ]:
state = envs.observation_space.sample()
state.shape, type(state)

((84, 84, 4), numpy.ndarray)

In [ ]:
dqn = DQN_Network(envs.observation_space.shape, envs.action_space.n)

In [ ]:
dqn(state)

torch.Size([1, 4, 84, 84]) <class 'torch.Tensor'>


tensor([[-0.1129, -2.5360, -2.8048, -0.0035]], grad_fn=<AddmmBackward0>)

# Replay Buffer

In [ ]:
# Taken from https://github.com/sfujim/TD3/blob/master/utils.py

class ReplayBuffer(object):
	def __init__(self, state_dim, action_dim, max_size=int(1e6)):
		self.max_size = max_size
		self.ptr = 0
		self.size = 0

		self.state = np.zeros((max_size, state_dim))
		self.action = np.zeros((max_size, action_dim))
		self.next_state = np.zeros((max_size, state_dim))
		self.reward = np.zeros((max_size, 1))
		self.not_done = np.zeros((max_size, 1))

		self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


	def add(self, state, action, next_state, reward, done):
		self.state[self.ptr] = state
		self.action[self.ptr] = action
		self.next_state[self.ptr] = next_state
		self.reward[self.ptr] = reward
		self.not_done[self.ptr] = 1. - done

		self.ptr = (self.ptr + 1) % self.max_size
		self.size = min(self.size + 1, self.max_size)


	def sample(self, batch_size):
		ind = np.random.randint(0, self.size, size=batch_size)

		return (
			torch.FloatTensor(self.state[ind]).to(self.device),
			torch.FloatTensor(self.action[ind]).to(self.device),
			torch.FloatTensor(self.next_state[ind]).to(self.device),
			torch.FloatTensor(self.reward[ind]).to(self.device),
			torch.FloatTensor(self.not_done[ind]).to(self.device)
		)

# Config

In [ ]:
config_cnn = {
    'env': 'Breakout-v0',
    'total_timesteps': 300000,
    'replay_buffer_size': 5000,
    'minimum_replay_before_updates': 1000,
    'target_update_steps': 1000,
    'epsilon_start': 1.0,
    'epsilon_end': 0.01,
    'exploration_percentage': 0.9,
    'hidden': 128,
    'batch_size': 64,
    'learning_rate': 0.0001,
    'gamma': 0.99,
    'print_steps': 10000,
    'tensorboard_folder': './tensorboard/',
    'checkpointing_steps': 10000,
    'biased_exploration': True,
    'seed': 12345,
}


In [ ]:
envs.action_space.sample()

1

# DQN main class

In [ ]:
import time, datetime,os

class DQN():

    def __init__(self, config):
        super(DQN).__init__()

        self.config = config

        # init the env
        self.env = VecFrameStack(
        DummyVecEnv([make_env(config['env'], 0)]),4)

        self.action_dim = self.env.action_space.n
        if type(self.env.observation_space) is gym.spaces.discrete.Discrete:
            self.state_dim = self.env.observation_space.n
        else:
            self.state_dim = self.env.observation_space.shape[0]

        # init the replay buffer
        self.buffer = ReplayBuffer(self.state_dim, self.action_dim, self.config['replay_buffer_size'])

        # Setup the DQN and Target networks and the syncing mechanism
        self.dqn = DQN_Network(self.state_dim, self.action_dim)
        self.target = DQN_Network(self.state_dim, self.action_dim)
        # Sync target and dqn networks
        self.update_target_network()

        # setup the optimiser: 
        self.opt = torch.optim.Adam(self.dqn.parameters(),lr=config['learning_rate'])

        # Setup the tensorboard writer
        self.setup_logging()
        self.epsilon_setup()

    def setup_logging(self):
        ts = time.time()
        self.timestamp = datetime.datetime.fromtimestamp(ts).strftime('%Y%m%d%H%M%S')

        log_dir = self.config['tensorboard_folder']+self.timestamp
        if not os.path.exists(log_dir):
            os.makedirs(log_dir)
        self.summary_writer = SummaryWriter(log_dir=log_dir, flush_secs=30)

    def epsilon_setup(self):
        self.duration = self.config['total_timesteps'] * self.config['exploration_percentage']
        self.start_e = self.config['epsilon_start']
        self.end_e = self.config['epsilon_end']
        self.slope =  (self.end_e - self.start_e) / self.duration
        self.epsilon = self.start_e

    def epsilon_linear_schedule(self, t: int):
        # borrowed from cleanrl just cause it was neater...
        return max(self.slope * t + self.start_e, self.end_e)

    def select_action(self, state):
        if np.random.random() < self.epsilon:
          action = torch.tensor(self.env.action_space.sample(), dtype=torch.int64)
        else:
          with torch.no_grad():
              action = self.dqn(state).argmax()  # n_batch (= 1) x n_actions

        return action.item()

    def update_target_network(self):
        # copy network params from DQN to Target NN
        self.target.load_state_dict(self.dqn.state_dict())

    def calculate_targets(self, next_states, rewards, not_dones):

        next_action_values = self.target(next_states)
        targets = rewards + self.config['gamma'] * not_dones * (
            torch.max(next_action_values.detach(),dim=1).values)

        return targets.unsqueeze(-1)  ### Stopped here

    def get_batch_from_buffer(self):
        # sample a batch and return as tensors
        states, next_states, actions, rewards, dones = self.buffer.sample(self.config['batch_size'])
        states_t = torch.tensor(states, dtype=torch.float)
        next_states_t = torch.tensor(states, dtype=torch.float)
        actions_t = torch.tensor(actions, dtype=torch.int)
        rewards_t = torch.tensor(rewards, dtype=torch.float)
        dones_t = torch.tensor(dones, dtype=torch.bool)

        return states_t, actions_t, next_states_t, rewards_t, dones_t

    def log_network_stats(self):
        # capture the grad norm to tensorboard
        pass

    def mse_loss(self, states, actions, targets):
        # get q-values - forward predictions
        current_q_values = self.dqn.forward(states)

        # print("all q_values: ", current_q_values.detach().shape)
        # print("actions: ", len(actions), actions)
        actions_t = torch.tensor(actions, dtype=torch.int64).unsqueeze(-1)
        q_values = torch.gather(current_q_values, -1, actions_t)
        # MSE loss
        loss = F.mse_loss(q_values, targets)

        self.opt.zero_grad()
        loss.backward()
        self.opt.step()

        self.log_network_stats()

        return loss.item()

    def save(self, filename):
        torch.save(self.dqn.state_dict(), filename+"_"+str(self.timestamp))

    def train(self):
        total_steps = 0
        episode_counter = 0
        state = self.env.reset()

        ep_rewards = []
        ep_len = 0

        running_loss = 0.0

        # With a max step count limit, run episodes, collect data and update the DQN.
        # Outside loop counts max steps for DQN in total. Could also set this to a max episode count
        # When do we perform the update: once per step (not waiting for episode completion or anything)

        while total_steps < self.config['total_timesteps']:

            # Collect data until the buffer has reached the minimum size
            self.epsilon = self.epsilon_linear_schedule(total_steps)
            self.summary_writer.add_scalar("epsilon", self.epsilon, total_steps)

            action = self.select_action(state)
            next_state, reward, done, info = self.env.step(action)

            # collect data
            self.buffer.insert_datapoint(state, action, next_state, reward, done)

            if self.buffer.len() < self.config['minimum_replay_before_updates']:
                if done:
                    state = self.env.reset()
                else:
                    state = next_state
                continue

            # When do we start logging the rewards and lengths? After we have min buffer info?
            ep_rewards.append(reward)
            ep_len +=1

            # Update the target network from the dqn's latest weights:
            if total_steps % self.config['target_update_steps'] ==0:
                self.update_target_network()

            # If we have enough data in the replay buffer, start training a batch after every step is collected from
            # the end, i.e. update the DQN
            # 1. sample a batch from the buffer
            states, next_states, actions, rewards, dones = self.buffer.sample(self.config['batch_size'])

            # 2. calculate the target q-values
            targets = self.calculate_targets(next_states, rewards, dones)

            # 3. Calculate MSE loss
            loss = self.mse_loss(states, actions, targets)  # returns loss.item()
            running_loss += loss
            self.summary_writer.add_scalar("loss", loss, total_steps-self.config['minimum_replay_before_updates'])
            self.summary_writer.add_scalar("running_loss", running_loss, total_steps-self.config[
                'minimum_replay_before_updates'])

            # set state to next_state
            state = next_state

            # update step counter
            total_steps+=1

            # Print some stats to screen
            if total_steps % self.config['print_steps'] == 0:
                print("Loss at ", total_steps, " steps is ", loss)

            if total_steps % self.config['checkpointing_steps'] == 0:
                self.save("chkpts/chkpt_dqn_"+str(total_steps/self.config['checkpointing_steps']))

            # process episode
            if done:
                state = self.env.reset()
                self.summary_writer.add_scalar("episode_reward", sum(ep_rewards), episode_counter)
                self.summary_writer.add_scalar("episode_length", ep_len, episode_counter)
                episode_counter +=1
                ep_rewards = []
                ep_len = 0

            self.summary_writer.flush()

        self.summary_writer.close()
        self.save("chkpts/final_chkpt_dqn")

